<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       In-Database Complaints Analysis Integration with Customer360 using LLMs
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Complaints Analysis Integration with Customer360 is a comprehensive approach to managing customer complaints and feedback within the framework of a Customer 360-degree view using <b>Teradata Vantage</b> and <b>Amazon Bedrock</b>. This integration aims to provide a seamless and personalized customer experience by leveraging data from various sources, including CRM systems, marketing platforms, and social media.</p> <p style='font-size:16px;font-family:Arial;color:#00233C'>The key components of this integration include:</p> 

<ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Customer 360 Data Manager</b>: Responsible for managing and maintaining a comprehensive view of customer data, including collection, integration, and analysis from multiple sources.</li> <li><b>Complaints Management Dashboard</b>: Analyzes customer complaints, providing insights into complaint volume, trends, and resolution progress.</li> <li><b>Customer Insights</b>: Tools for gaining insights into customer behavior and preferences, enabling targeted marketing campaigns and informed business decisions.</li> </ol> <p style='font-size:16px;font-family:Arial;color:#00233C'>The benefits of this integration include:</p> <ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Improved Customer Experience</b>: By integrating complaints analysis with Customer 360, businesses can address customer complaints more effectively, leading to increased customer satisfaction and loyalty.</li> <li><b>Data-Driven Decision Making</b>: The integration provides a centralized platform for analyzing customer data, enabling businesses to make informed decisions about product development, marketing strategies, and customer engagement.</li> <li><b>Enhanced Customer Insights</b>: The comprehensive view of customer data allows businesses to better understand customer needs and preferences, leading to more targeted and effective marketing efforts.</li> </ol> 


<p style='font-size:16px;font-family:Arial;color:#00233C'>By integrating complaints analysis with Customer 360, businesses can create a more comprehensive and personalized customer experience, driving business growth and customer satisfaction.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Connect to Vantage</li>
    <li>Configure server-side LLM access using Teradatagenai package</li>
    <li>Execute in-database Sentiment Analysis, Topic Modelling and Complaint Summarization</li>
    <li>Integrate data with customer 360</li>
    <li>Cleanup</li>
</ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Download and install additional software as needed</b>

In [ ]:
%pip install --upgrade -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [16]:
import numpy as np
import pandas as pd
import json, warnings

from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi



# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
display.suppress_vantage_runtime_warnings = True
display.max_rows = 5

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Connection information has been defined in an external file - adjust as necessary.</p>

In [10]:
# load vars json
with open('vars-vs_demo.json', 'r') as f:
    session_vars = json.load(f)

# Database login information
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

# connect to database
eng = create_context(host = host, username = username, password = password)


Authentication token is generated, authenticated and set for the session.


<hr>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key_id: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

In [17]:
# Provide model details
model_name = 'anthropic.claude-instant-v1'

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)


obj = TextAnalyticsAI(llm=llm)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Using LLM for Sentiment Analysis, Topic Modeling and Complaint Summarization</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis, Topic Modeling and Complaint Summarization using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis Using Large Language Models (LLMs) is a cutting-edge approach to understanding customer opinions and emotions expressed through text-based data. This advanced technique leverages the capabilities of LLMs to accurately identify and categorize sentiment as positive, negative, or neutral, providing businesses with valuable insights into customer perceptions and preferences.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Inspect source data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata python package (teradataml) allows users to work with data using common python syntax and methods <b>without</b> moving data to the client - all operations are pushed to the MPP platform, allowing rapid, performant analytics on data at any scale.  In this case, the DataFrame object represents a table or query <b>in-database</b> which could contain millions or billions of records.</p>

In [11]:
customer_data = DataFrame('"demo"."Customer_360_Details"')
customer_data

Customer Identifier,Name,City,State,Customer Type,Product Holdings,Total Deposit Balance,Total Credit Balance,Total Investments AUM,Customer Profitability,Customer Lifetime Value,Bank Tenure,Affluence Segment,Digital Banking Segment,Branch Banking Segment
456789123,Alphonse S. Mercado,West Covina,CA,Consumer & Business,"Checking, Home Equity, Brokerage",23291,122422,68369,312,3712,10.2,Private Banking,Not Online,Branch only
123456789,John Q. Public,Rancho Bernardo,CA,Consumer Only,"Checking, Savings, Credit Card",1425,5241,0,105,425,4.5,Mass Affluent,Digital Monetary,Branch & ATM
789456123,Linda H. Gomes,Fife,WA,Consumer Only,Mortgage,0,272162,0,225,1865,3.8,Mass Market,Not Online,nan
135724681,Cardi B. Jones,Syosset,NY,Consumer Only,Credit Card,0,697,0,33,89,1.4,Mass Market,Digital Access,nan
987654321,Jane Z. Doe,El Cerrito,CA,Consumer Only,"Checking, Savings",825,0,0,42,107,6.3,Mass Market Digital Access,ATM only,nan


In [12]:
complaints_data = DataFrame(in_schema('demo', 'Customer_360_Complaints'))
complaints_data

date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id,Customer_ID
19/12/12,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,nan,"I had a home equity loan with Discover Bank for {$75000.00}. In both the Loan Estimate and Closing Disclosure, it states under "" Prepayment Penalty '' that there was none. I was also told this verbally. However, in a later set of documents it does describe a prepayment penalty if paid off within 36 months. I believe this discrepancy between the documents is deceptive. I believe I was lead to believe that there would not be a prepayment penalty and was surprised to find this {$420.00} penalty in my payoff letter. I spoke with a representative who agreed that it was misleading then a supervisor who noted that it was in another document which I signed. I told him I felt deceived but he was not willing to refund the prepayment penalty. I'm curious as to how many other people they are doing this to.",**********************,DISCOVER BANK,IL,60302,**********************,Consent provided,Web,19/12/12,Closed with monetary relief,Yes,*****,3466832,456789123
23/09/04,Mortgage,Home equity loan or line of credit (HELOC),Applying for a mortgage or refinancing an existing mortgage,Application denials,I went to apply for XXXX home equity loan on XX/XX/XXXX. I got full pre-approval under the terms of 30 year loan interest of 9.99 %. I accepted this. They then requested lots of different information that I submitted. Every file said accepted. On XX/XX/XXXX I called them up because there was a small problem with the mortage statement I submitted. At this time I was told I was no longer approved because of debt to income ratio. The thing being I already had pre-approval and even the proof of income was already accepted. This loan would have been XXXX of my current debt payments without the loan.,**********************,DISCOVER BANK,WY,82801,**********************,Consent provided,Web,23/09/04,Closed with explanation,Yes,*****,7494116,135724681
21/05/11,Mortgage,Conventional home mortgage,Closing on a mortgage,nan,"Mortgage was paid off on XX/XX/2021. Home was sold, we moved to Florida. We are nearly 60 days and the escrow refund has not been issued.",**********************,DISCOVER BANK,FL,328XX,**********************,Consent provided,Web,21/05/11,Closed with monetary relief,Yes,*****,4368877,789456123
19/12/02,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,nan,"Hello, I had a second mortgage with Discover and sold the property. The second had a pre-payment penalty before making 12 payments starting XX/XX/XXXX. The property was sold in XX/XX/XXXX. I recieved a mortgage statement for XX/XX/XXXX that clearly states no pre-payment penalty on the statement. I was charged a {$480.00} pre-payment penalty. I called discover and they told me that if I was refinancing I would not have a penalty. But selling the property I do have penalty.??? My paperwork says no pre-payment penalty after 12 months of payments. Which I did make. This is rubbish trying to pass the buck based on tricky language. Either you have a pre-payment penalty or you do not. I did as my paperwork said, and reflected on my most recent statement. Please send payment to me accordingly.",**********************,DISCOVER BANK,FL,XXXXX,**********************,Consent provided,Web,19/12/02,Closed with explanation,Yes,*****,3455874,987654321
22/12/04,Mortgage,Home equity loan or line of credit (HELOC),Applying for a mortgage or refinancing an existing mortgage,nan,"I received a postcard from XXXX about a Home Equity Loan. While I was entering information a message came up they were running my credit. I never checked a box to authorize I wanted to complete the application nor have my credit

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Sentiment Analysis
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Extract the sentiment (positive, negative, neutral) using in-database functions that can execute in-platform or call out to Large Language Models of choice.</p>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Extract Customer Sentiment</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A simple method call will extract the sentiment for patient comments <b>in-database</b> using the desired LLM and CSP provider.</p>

In [60]:
tdf_sentiment = obj.analyze_sentiment(column = 'consumer_complaint_narrative', 
                                      data = complaints_data)[['Customer_ID','Sentiment','consumer_complaint_narrative']]
tdf_sentiment

Customer_ID,Sentiment,consumer_complaint_narrative
123456789,negative,"I received a postcard from XXXX about a Home Equity Loan. While I was entering information a message came up they were running my credit. I never checked a box to authorize I wanted to complete the application nor have my credit run. I was not applying I was only trying to get information. I tried to stop by pushing the back button and then I got a message from XXXX XXXX stating a new credit inquiry was made on my credit. I never got any confirmation email from Discover and at first thought it was a fraudulent website. I am requesting the credit inquiry be removed from my credit. The Discover representative said all she could do was close out the application. I have been a Discover customer from many years and this totally destroyed any trust I had in this company. Sincerely, XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX FL XXXX"
135724681,negative,I went to apply for XXXX home equity loan on XX/XX/XXXX. I got full pre-approval under the terms of 30 year loan interest of 9.99 %. I accepted this. They then requested lots of different information that I submitted. Every file said accepted. On XX/XX/XXXX I called them up because there was a small problem with the mortage statement I submitted. At this time I was told I was no longer approved because of debt to income ratio. The thing being I already had pre-approval and even the proof of income was already accepted. This loan would have been XXXX of my current debt payments without the loan.
456789123,negative,"I had a home equity loan with Discover Bank for {$75000.00}. In both the Loan Estimate and Closing Disclosure, it states under "" Prepayment Penalty '' that there was none. I was also told this verbally. However, in a later set of documents it does describe a prepayment penalty if paid off within 36 months. I believe this discrepancy between the documents is deceptive. I believe I was lead to believe that there would not be a prepayment penalty and was surprised to find this {$420.00} penalty in my payoff letter. I spoke with a representative who agreed that it was misleading then a supervisor who noted that it was in another document which I signed. I told him I felt deceived but he was not willing to refund the prepayment penalty. I'm curious as to how many other people they are doing this to."
987654321,negative,"Hello, I had a second mortgage with Discover and sold the property. The second had a pre-payment penalty before making 12 payments starting XX/XX/XXXX. The property was sold in XX/XX/XXXX. I recieved a mortgage statement for XX/XX/XXXX that clearly states no pre-payment penalty on the statement. I was charged a {$480.00} pre-payment penalty. I called discover and they told me that if I was refinancing I would not have a penalty. But selling the property I do have penalty.??? My paperwork says no pre-payment penalty after 12 months of payments. Which I did make. This is rubbish trying to pass the buck based on tricky language. Either you have a pre-payment penalty or you do not. I did as my paperwork said, and reflected on my most recent statement. Please send payment to me accordingly."
789456123,negative,"Mortgage was paid off on XX/XX/2021. Home was sold, we moved to Florida. We are nearly 60 days and the escrow refund has not been issued."


<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Topic Modeling
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts. In this case, we are looking for specific topics to drive downstream analytics.</p>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Use the Model to perform topic modeling</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Provide a list of topics to use for classification.</p>

In [61]:
tdf_topics = obj.classify(column = 'consumer_complaint_narrative', 
                          data = complaints_data,
                          labels = ['Mortgage Application',
                                    'Payment Trouble',
                                    'Mortgage Closing',
                                    'Report Inaccuracy',
                                    'Payment Struggle'])[['Customer_ID','Labels','consumer_complaint_narrative']]
tdf_topics

Customer_ID,Labels,consumer_complaint_narrative
123456789,Report Inaccuracy,"I received a postcard from XXXX about a Home Equity Loan. While I was entering information a message came up they were running my credit. I never checked a box to authorize I wanted to complete the application nor have my credit run. I was not applying I was only trying to get information. I tried to stop by pushing the back button and then I got a message from XXXX XXXX stating a new credit inquiry was made on my credit. I never got any confirmation email from Discover and at first thought it was a fraudulent website. I am requesting the credit inquiry be removed from my credit. The Discover representative said all she could do was close out the application. I have been a Discover customer from many years and this totally destroyed any trust I had in this company. Sincerely, XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX FL XXXX"
135724681,Payment Trouble,I went to apply for XXXX home equity loan on XX/XX/XXXX. I got full pre-approval under the terms of 30 year loan interest of 9.99 %. I accepted this. They then requested lots of different information that I submitted. Every file said accepted. On XX/XX/XXXX I called them up because there was a small problem with the mortage statement I submitted. At this time I was told I was no longer approved because of debt to income ratio. The thing being I already had pre-approval and even the proof of income was already accepted. This loan would have been XXXX of my current debt payments without the loan.
456789123,Payment Struggle,"I had a home equity loan with Discover Bank for {$75000.00}. In both the Loan Estimate and Closing Disclosure, it states under "" Prepayment Penalty '' that there was none. I was also told this verbally. However, in a later set of documents it does describe a prepayment penalty if paid off within 36 months. I believe this discrepancy between the documents is deceptive. I believe I was lead to believe that there would not be a prepayment penalty and was surprised to find this {$420.00} penalty in my payoff letter. I spoke with a representative who agreed that it was misleading then a supervisor who noted that it was in another document which I signed. I told him I felt deceived but he was not willing to refund the prepayment penalty. I'm curious as to how many other people they are doing this to."
987654321,Payment Trouble,"Hello, I had a second mortgage with Discover and sold the property. The second had a pre-payment penalty before making 12 payments starting XX/XX/XXXX. The property was sold in XX/XX/XXXX. I recieved a mortgage statement for XX/XX/XXXX that clearly states no pre-payment penalty on the statement. I was charged a {$480.00} pre-payment penalty. I called discover and they told me that if I was refinancing I would not have a penalty. But selling the property I do have penalty.??? My paperwork says no pre-payment penalty after 12 months of payments. Which I did make. This is rubbish trying to pass the buck based on tricky language. Either you have a pre-payment penalty or you do not. I did as my paperwork said, and reflected on my most recent statement. Please send payment to me accordingly."
789456123,Mortgage Closing,"Mortgage was paid off on XX/XX/2021. Home was sold, we moved to Florida. We are nearly 60 days and the escrow refund has not been issued."


<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Summarization
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The summarize method uses the model to summarize the text in the specified column of a database table. It generates an abstractive summary for the input using different levels. The conciseness of the summary can be adjusted using different levels. Higher levels yield more concise summaries.</p>

In [62]:
tdf_summary = obj.summarize(column = 'consumer_complaint_narrative', 
                          data = complaints_data,
                          levels = 2 # higher values provide more concise summary
                           )[['Customer_ID','Summary','consumer_complaint_narrative']]
tdf_summary

Customer_ID,Summary,consumer_complaint_narrative
123456789,"A postcard for a Home Equity Loan automatically ran my credit without consent, causing a credit inquiry notice and lost trust in the company.","I received a postcard from XXXX about a Home Equity Loan. While I was entering information a message came up they were running my credit. I never checked a box to authorize I wanted to complete the application nor have my credit run. I was not applying I was only trying to get information. I tried to stop by pushing the back button and then I got a message from XXXX XXXX stating a new credit inquiry was made on my credit. I never got any confirmation email from Discover and at first thought it was a fraudulent website. I am requesting the credit inquiry be removed from my credit. The Discover representative said all she could do was close out the application. I have been a Discover customer from many years and this totally destroyed any trust I had in this company. Sincerely, XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX FL XXXX"
135724681,"I applied for and was initially approved for a home equity loan, but the approval was later revoked due to my debt-to-income ratio despite submitting required documents.",I went to apply for XXXX home equity loan on XX/XX/XXXX. I got full pre-approval under the terms of 30 year loan interest of 9.99 %. I accepted this. They then requested lots of different information that I submitted. Every file said accepted. On XX/XX/XXXX I called them up because there was a small problem with the mortage statement I submitted. At this time I was told I was no longer approved because of debt to income ratio. The thing being I already had pre-approval and even the proof of income was already accepted. This loan would have been XXXX of my current debt payments without the loan.
456789123,The customer was charged a $420 prepayment penalty for paying off a home equity loan within 36 months even though the Loan Estimate and Closing Disclosure from Discover Bank did not disclose such a penalty.,"I had a home equity loan with Discover Bank for {$75000.00}. In both the Loan Estimate and Closing Disclosure, it states under "" Prepayment Penalty '' that there was none. I was also told this verbally. However, in a later set of documents it does describe a prepayment penalty if paid off within 36 months. I believe this discrepancy between the documents is deceptive. I believe I was lead to believe that there would not be a prepayment penalty and was surprised to find this {$420.00} penalty in my payoff letter. I spoke with a representative who agreed that it was misleading then a supervisor who noted that it was in another document which I signed. I told him I felt deceived but he was not willing to refund the prepayment penalty. I'm curious as to how many other people they are doing this to."
987654321,"The property owner requested reimbursement of a $480 prepayment penalty charged by Discover, though recent documents showed no such penalty should have been applied after making timely payments for 12 months.","Hello, I had a second mortgage with Discover and sold the property. The second had a pre-payment penalty before making 12 payments starting XX/XX/XXXX. The property was sold in XX/XX/XXXX. I recieved a mortgage statement for XX/XX/XXXX that clearly states no pre-payment penalty on the statement. I was charged a {$480.00} pre-payment penalty. I called discover and they told me that if I was refinancing I would not have a penalty. But selling the property I do have penalty.??? My paperwork says no pre-payment penalty after 12 months of payments. Which I did make. This is rubbish trying to pass the buck based on tricky language. Either you have a pre-payment penalty or you do not. I did as my paperwork said, and reflected on my most recent statement. Please send payment to me accordingly."
789456123,The homeowners are still waiting for their escrow refund over two months after paying off their mortgage and selling the

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Consolidated data in-database Customer360
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The developer can now perform simple joins on the data <b>in-database</b> to provide a consolidated view of the complaint summary, sentiment, topic label, and customer information.</p>

In [67]:
tdf_combined = customer_data.join(tdf_topics.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier" = Customer_ID']).drop('Customer_ID', axis = 1)
tdf_combined = tdf_combined.join(tdf_summary.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier" = Customer_ID']).drop('Customer_ID', axis = 1)
tdf_combined = tdf_combined.join(tdf_sentiment.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier"= Customer_ID']).drop('Customer_ID', axis = 1)

In [69]:
tdf_combined[['Customer Identifier','Name','Sentiment','Labels','Summary']]

Customer Identifier,Name,Sentiment,Labels,Summary
987654321,Jane Z. Doe,negative,Payment Trouble,"The property owner requested reimbursement of a $480 prepayment penalty charged by Discover, though recent documents showed no such penalty should have been applied after making timely payments for 12 months."
456789123,Alphonse S. Mercado,negative,Payment Struggle,The customer was charged a $420 prepayment penalty for paying off a home equity loan within 36 months even though the Loan Estimate and Closing Disclosure from Discover Bank did not disclose such a penalty.
135724681,Cardi B. Jones,negative,Payment Trouble,"I applied for and was initially approved for a home equity loan, but the approval was later revoked due to my debt-to-income ratio despite submitting required documents."
123456789,John Q. Public,negative,Report Inaccuracy,"A postcard for a Home Equity Loan automatically ran my credit without consent, causing a credit inquiry notice and lost trust in the company."
789456123,Linda H. Gomes,negative,Mortgage Closing,The homeowners are still waiting for their escrow refund over two months after paying off their mortgage and selling their home to relocate to Florida.


<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Persist the dataset</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple python methods will materialize the data to a permanent table if desired.</p>

In [71]:
copy_to_sql(tdf_combined, table_name = 'Customer360', temporary = True, if_exists = 'replace')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025. All Rights Reserved
        </div>
    </div>
</footer>